# DS-SF-34 | 03 | Databases, Scrapping, and APIs | Assignment | Answer Key

## The Bistro Meets `SQLite`

In this assignment, we will be exploring the `bistro` dataset.  The previous assignment used `pandas`.  Today, we'll answer the same questions but using `SQLite`.  In some situations, `pandas` will be a better solution.  In others, doing it using `SQL` will make more sense.  As you gain more experience, you'll know which one to use.

> ### Question 1.  Import the `sqlite3` package.

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### Question 2.  Connect to the `dataset-03-bistro.db` database.  The rest of this assignment focus on the `bistro` table.

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-03-bistro.db'))

> ### Question 3.  How many samples (i.e., rows) are in this dataset?

In [3]:
pd.io.sql.read_sql(
'''
SELECT COUNT()
    FROM bistro
    LIMIT 10
;
''', con = db)

,COUNT()
0,244


Answer: 244

> ### Question 4.  Print the first two rows of the table to the console.

In [4]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM bistro
    LIMIT 2
;
''', con = db)

,index,day,time,name,gender,is_smoker,party,check,tip
0,0,Sunday,Dinner,Kimberly,Female,0,2,16.99,1.01
1,1,Sunday,Dinner,Nicholas,Male,0,3,10.34,1.66


> ### Question 5.  For which week days does the dataset has data for?

In [5]:
pd.io.sql.read_sql(
'''
SELECT DISTINCT day
    FROM bistro
    ORDER BY day
;
''', con = db)

,day
0,Friday
1,Saturday
2,Sunday
3,Thursday


Answer: Thursdays to Sundays.  (No data for Mondays to Wednesdays)

> ### Question 6.  How often was the bistro patronized for each week day?

In [6]:
pd.io.sql.read_sql(
'''
SELECT day, COUNT() as total_visits
    FROM bistro
    GROUP BY day
    ORDER BY day
;
''', con = db)

,day,total_visits
0,Friday,19
1,Saturday,87
2,Sunday,76
3,Thursday,62


Answer:

| Day | Total Visits |
|:---:|:---:|
| Thursdays | 62 |
| Fridays | 19 |
| Saturdays | 87 |
| Sundays | 76 |

> ### Question 7.  How much tip did waiters collect for each week day?

In [7]:
pd.io.sql.read_sql(
'''
SELECT day, SUM(tip) as total_tips
    FROM bistro
    GROUP BY day
    ORDER BY day
;
''', con = db)

,day,total_tips
0,Friday,51.96
1,Saturday,260.40
2,Sunday,247.39
3,Thursday,171.83


Answer:

| Day | Total Tips |
|:---:|:---:|
| Thursdays | \$171.83 |
| Fridays | \$51.96 |
| Saturdays | \$260.40 |
| Sundays | \$247.39 |

> ### Question 8.  What is the average tip per check (in absolute \$) for each week day?

In [8]:
pd.io.sql.read_sql(
'''
SELECT day, SUM(tip) / COUNT() as average_tip_per_check
    FROM bistro
    GROUP BY day
    ORDER BY day
;
''', con = db)

,day,average_tip_per_check
0,Friday,2.734737
1,Saturday,2.993103
2,Sunday,3.255132
3,Thursday,2.771452


Answer:

| Day | Average tip per check |
|:---:|:---:|
| Thursdays | \$2.77 |
| Fridays | \$2.73 |
| Saturdays | \$2.99 |
| Sundays | \$3.25 |

> ### Question 9.  What is the average tip per check (as a percentage of the check) for each week day?

(`CHECK` is a reserved keywork; use `` `check` `` (put the name between backticks) to reference the `check` column)

In [9]:
pd.io.sql.read_sql(
'''
SELECT day, SUM(tip) / SUM(`check`) as average_tip_per_check
    FROM bistro
    GROUP BY day
    ORDER BY day
;
''', con = db)

,day,average_tip_per_check
0,Friday,0.159445
1,Saturday,0.146424
2,Sunday,0.152038
3,Thursday,0.156732


Answer:

| Day | Average tip per check |
|:---:|:---:|
| Thursdays | 15.7% |
| Fridays | 15.9% |
| Saturdays | 14.6% |
| Sundays | 15.2% |

> ### Question 10.  Are there any name in common between male and female patrons?  (E.g., `Chris` can refer to either a man or a woman)

(check `numpy.intersect1d()`; it could come in handy)

(https://docs.scipy.org/doc/numpy/reference/generated/numpy.intersect1d.html)

In [10]:
pd.io.sql.read_sql(
'''
SELECT DISTINCT name
    FROM bistro
    WHERE gender = 'Male'
INTERSECT
SELECT DISTINCT name
    FROM bistro
    WHERE gender = 'Female'
;
''', con = db)

,name
0,Casey


Answer: `Casey` is a name used by both a male and a female patron.

> ### Question 11.  If no patrons share the same name, how many unique patrons are in the dataset?

In [11]:
pd.io.sql.read_sql(
'''
WITH names_by_gender AS
    (SELECT name, gender
        FROM bistro
        GROUP BY name, gender)

SELECT COUNT()
    FROM names_by_gender
;
''', con = db)

,COUNT()
0,182


(or)

In [12]:
pd.io.sql.read_sql(
'''
SELECT COUNT()
    FROM (SELECT name, gender
        FROM bistro
        GROUP BY name, gender)
;
''', con = db)

,COUNT()
0,182


Answer: 182

> ### Question 12.  How many times did `Kevin` patronized the bistro?  How about `Alice`?

In [13]:
pd.io.sql.read_sql(
'''
SELECT COUNT()
    FROM bistro
    WHERE name = 'Kevin'
;
''', con = db)

,COUNT()
0,4


In [14]:
pd.io.sql.read_sql(
'''
SELECT COUNT()
    FROM bistro
    WHERE name = 'Alice'
;
''', con = db)

,COUNT()
0,2


Answer:
- `Kevin`: 4
- `Alice`: 2

> ### Question 13.  Who are the top 3 female and male patrons?

In [15]:
pd.io.sql.read_sql(
'''
WITH visits AS
    (SELECT name, gender, COUNT() as visits
        FROM bistro
        GROUP BY name, gender
        ORDER BY visits DESC),

top_female_patrons AS
    (SELECT *
        FROM visits
        WHERE gender = 'Female'
        LIMIT 3),

top_male_patrons AS
    (SELECT *
        FROM visits
        WHERE gender = 'Male'
        LIMIT 3)

SELECT * from top_female_patrons
UNION ALL
SELECT * from top_male_patrons
;
''', con = db)

,name,gender,visits
0,Mary,Female,4
1,Casey,Female,3
2,Laura,Female,3
3,David,Male,8
4,Casey,Male,5
5,James,Male,5


Answer:

- Top 3 women: `Mary` (4); `Laura` and `Casey` (3 each)
- Top 3 men: `David` (8); `James` and `Casey` (5 each)

> ### Question 14.  Who's the best tipper (as a fraction of all tips over all check totals)?  Who's the worst?  How many times did they patronize the bistro?

In [16]:
pd.io.sql.read_sql(
'''
SELECT name, SUM(tip) / SUM(`check`) as average_tip_per_check, COUNT() as visits
    FROM bistro
    GROUP BY name, gender
    ORDER BY average_tip_per_check DESC, name, gender
    LIMIT 1
;
''', con = db)

,name,average_tip_per_check,visits
0,Maryann,0.416667,1


In [17]:
pd.io.sql.read_sql(
'''
SELECT name, SUM(tip) / SUM(`check`) as average_tip_per_check, COUNT() as visits
    FROM bistro
    GROUP BY name, gender
    ORDER BY average_tip_per_check, name, gender
    LIMIT 1
;
''', con = db)

,name,average_tip_per_check,visits
0,Jeremy,0.035638,1


Answer:
- `Maryann` is the best tipper.  She patronized the restaurant once.
- `Jeremy` is on the over end the worst tipper.  He patronized the restaurant also only once.